In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torch.utils.data import Dataset, DataLoader
import math
import sys
import cmath

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from torchvision import datasets
import torchvision.transforms as transforms

import argparse
import time

In [ ]:
class JetTrainData(Dataset):
    
    def __init__(self):

        self.x = torch.from_numpy( np.load('/data/github/data/diHiggs_neutrino_train_data.npy').astype(np.float32) )
        #                                               hh               tt             tw             tth            ttv            llbj           tatabb
        self.y = torch.from_numpy( np.concatenate((np.ones(19000), np.zeros(9900), np.zeros(5500), np.zeros(200), np.zeros(250), np.zeros(1000), np.zeros(60))).astype(np.int) )
        self.n_samples = 35910
                
    def __getitem__(self,index):
        
        return self.x[index], self.y[index]
        
    def __len__(self):
        return self.n_samples

    
class JetTestData(Dataset):
    
    def __init__(self):

        self.x = torch.from_numpy( np.load('/data/github/data/diHiggs_neutrino_test_data.npy').astype(np.float32) )
        #                                               hh              tt              tw            tth            ttv            llbj          tatabb
        self.y = torch.from_numpy( np.concatenate((np.ones(5000), np.zeros(19000), np.zeros(800), np.zeros(100), np.zeros(100), np.zeros(240), np.zeros(10))).astype(np.int) )
        self.n_samples = 25250
                
    def __getitem__(self,index):
        
        return self.x[index], self.y[index]
        
    def __len__(self):
        return self.n_samples

In [ ]:
num_workers = 0

batch_size = 20

train_data = JetTrainData()
test_data = JetTestData()

train_loader = DataLoader(dataset=train_data, batch_size = batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(dataset=test_data, batch_size = batch_size, shuffle=False, num_workers=num_workers)

dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy()

fig = plt.figure(figsize=(25, 4))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(2, int(batch_size/2), idx+1, xticks=[], yticks=[])
    ax.imshow(images[idx,2], cmap='gray')
    ax.set_title(str(labels[idx].item()))

In [ ]:
class PrimaryCaps(nn.Module):
    def __init__(self, A, B, K, P, stride):
        super(PrimaryCaps, self).__init__()
        self.pose = nn.Conv2d(in_channels=A, out_channels=B*P*P,
                            kernel_size=K, stride=stride, bias=True)
        self.a = nn.Conv2d(in_channels=A, out_channels=B,
                            kernel_size=K, stride=stride, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        p = self.pose(x)
        a = self.a(x)
        a = self.sigmoid(a)
        out = torch.cat([p, a], dim=1)
        out = out.permute(0, 2, 3, 1)
        return out

In [ ]:
class ConvCaps(nn.Module):
    def __init__(self, B, C, K, P, stride, iters,
                 coor_add=False, w_shared=False):
        super(ConvCaps, self).__init__()
        self.B = B
        self.C = C
        self.K = K
        self.P = P
        self.psize = P*P
        self.stride = stride
        self.iters = iters
        self.coor_add = coor_add
        self.w_shared = w_shared
        self.eps = 1e-8
        self._lambda = 1e-03
        self.ln_2pi = torch.cuda.FloatTensor(1).fill_(math.log(2*math.pi))
        self.beta_u = nn.Parameter(torch.zeros(C))
        self.beta_a = nn.Parameter(torch.zeros(C))
        self.weights = nn.Parameter(torch.randn(1, K*K*B, C, P, P))
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=2)

    def m_step(self, a_in, r, v, eps, b, B, C, psize):
        r = r * a_in
        r = r / (r.sum(dim=2, keepdim=True) + eps)
        r_sum = r.sum(dim=1, keepdim=True)
        coeff = r / (r_sum + eps)
        coeff = coeff.view(b, B, C, 1)

        mu = torch.sum(coeff * v, dim=1, keepdim=True)
        sigma_sq = torch.sum(coeff * (v - mu)**2, dim=1, keepdim=True) + eps

        r_sum = r_sum.view(b, C, 1)
        sigma_sq = sigma_sq.view(b, C, psize)
        cost_h = (self.beta_u.view(C, 1) + torch.log(sigma_sq.sqrt())) * r_sum

        a_out = self.sigmoid(self._lambda*(self.beta_a - cost_h.sum(dim=2)))
        sigma_sq = sigma_sq.view(b, 1, C, psize)

        return a_out, mu, sigma_sq

    def e_step(self, mu, sigma_sq, a_out, v, eps, b, C):
        ln_p_j_h = -1. * (v - mu)**2 / (2 * sigma_sq) - torch.log(sigma_sq.sqrt()) - 0.5*self.ln_2pi

        ln_ap = ln_p_j_h.sum(dim=3) + torch.log(a_out.view(b, 1, C))
        r = self.softmax(ln_ap)
        
        return r

    def caps_em_routing(self, v, a_in, C, eps):
        b, B, c, psize = v.shape
        assert c == C
        assert (b, B, 1) == a_in.shape

        r = torch.cuda.FloatTensor(b, B, C).fill_(1./C)
        for iter_ in range(self.iters):
            a_out, mu, sigma_sq = self.m_step(a_in, r, v, eps, b, B, C, psize)
            if iter_ < self.iters - 1:
                r = self.e_step(mu, sigma_sq, a_out, v, eps, b, C)

        return mu, a_out

    def add_pathes(self, x, B, K, psize, stride):
        b, h, w, c = x.shape
        assert h == w
        assert c == B*(psize+1)
        oh = ow = int(((h - K )/stride)+ 1)
        idxs = [[(h_idx + k_idx) for k_idx in range(0, K)] for h_idx in range(0, h - K + 1, stride)]
        x = x[:, idxs, :, :]
        x = x[:, :, :, idxs, :]
        x = x.permute(0, 1, 3, 2, 4, 5).contiguous()
        
        return x, oh, ow

    def transform_view(self, x, w, C, P, w_shared=False):
        b, B, psize = x.shape
        assert psize == P*P

        x = x.view(b, B, 1, P, P)
        if w_shared:
            hw = int(B / w.size(1))
            w = w.repeat(1, hw, 1, 1, 1)

        w = w.repeat(b, 1, 1, 1, 1)
        x = x.repeat(1, 1, C, 1, 1)
        v = torch.matmul(x, w)
        v = v.view(b, B, C, P*P)
        return v

    def add_coord(self, v, b, h, w, B, C, psize):
        assert h == w
        v = v.view(b, h, w, B, C, psize)
        coor = torch.arange(h, dtype=torch.float32) / h
        coor_h = torch.cuda.FloatTensor(1, h, 1, 1, 1, self.psize).fill_(0.)
        coor_w = torch.cuda.FloatTensor(1, 1, w, 1, 1, self.psize).fill_(0.)
        coor_h[0, :, 0, 0, 0, 0] = coor
        coor_w[0, 0, :, 0, 0, 1] = coor
        v = v + coor_h + coor_w
        v = v.view(b, h*w*B, C, psize)
        
        return v

    def forward(self, x):
        b, h, w, c = x.shape
        if not self.w_shared:
            x, oh, ow = self.add_pathes(x, self.B, self.K, self.psize, self.stride)

            p_in = x[:, :, :, :, :, :self.B*self.psize].contiguous()
            a_in = x[:, :, :, :, :, self.B*self.psize:].contiguous()
            p_in = p_in.view(b*oh*ow, self.K*self.K*self.B, self.psize)
            a_in = a_in.view(b*oh*ow, self.K*self.K*self.B, 1)
            v = self.transform_view(p_in, self.weights, self.C, self.P)

            p_out, a_out = self.caps_em_routing(v, a_in, self.C, self.eps)
            p_out = p_out.view(b, oh, ow, self.C*self.psize)
            a_out = a_out.view(b, oh, ow, self.C)
            out = torch.cat([p_out, a_out], dim=3)
        else:
            assert c == self.B*(self.psize+1)
            assert 1 == self.K
            assert 1 == self.stride
            p_in = x[:, :, :, :self.B*self.psize].contiguous()
            p_in = p_in.view(b, h*w*self.B, self.psize)
            a_in = x[:, :, :, self.B*self.psize:].contiguous()
            a_in = a_in.view(b, h*w*self.B, 1)

            v = self.transform_view(p_in, self.weights, self.C, self.P, self.w_shared)

            if self.coor_add:
                v = self.add_coord(v, b, h, w, self.B, self.C, self.psize)

            _, out = self.caps_em_routing(v, a_in, self.C, self.eps)

        return out

In [ ]:
class CapsNet(nn.Module):
    def __init__(self, A, B, C, D, E, K, P, iters_routings):
        super(CapsNet, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=A,
                               kernel_size=(5,3,3), stride=1, padding=0)

        self.conv2 = nn.Conv2d(in_channels=A, out_channels=A,
                               kernel_size=(3), stride=2, padding=1)
        
        self.conv3 = nn.Conv2d(in_channels=A, out_channels=A,
                               kernel_size=(3), stride=1, padding=0)
        
        self.conv4 = nn.Conv2d(in_channels=A, out_channels=A,
                               kernel_size=(3), stride=1, padding=0)
        
        self.primary_caps = PrimaryCaps(A, B, 1, P, stride=1)

        self.conv_caps1 = ConvCaps(B, C, 5, P, stride=2, iters=iters_routings)

        self.conv_caps2 = ConvCaps(C, D, K, P, stride=1, iters=iters_routings)

        self.class_caps = ConvCaps(D, E, 1, P, stride=1, iters=iters_routings,
                                        coor_add=True, w_shared=True)
        
        print('=='*20)
        print('model info:','\nA, B, C, D, E, K, P =',str([A,B,C,D,E,K,P])
        ,'\niters for routings: ',str(iters_routings))
        print('=='*20)
        
    def forward(self, x):
        x = self.conv1(x.unsqueeze(1))
        x = (F.relu(x)).squeeze()
        x=self.conv2(x)
        x=F.relu(x)
        x=self.conv3(x)
        x=F.relu(x)
        x = self.primary_caps(x)
        x = self.conv_caps1(x)
        x = self.conv_caps2(x)
        x = self.class_caps(x)
        
        return x

In [ ]:
from torch.nn.modules.loss import _Loss

class SpreadLoss(_Loss):

    def __init__(self, m_min, m_max, num_class):
        super(SpreadLoss, self).__init__()
        self.m_min = m_min
        self.m_max = m_max
        self.num_class = num_class

    def forward(self, x, target, r):
        b, E = x.shape
        assert E == self.num_class
        margin = self.m_min + (self.m_max - self.m_min)*r

        at = torch.cuda.FloatTensor(b).fill_(0)
        for i, lb in enumerate(target):
            at[i] = x[i][lb]
        at = at.view(b, 1).repeat(1, E)

        zeros = x.new_zeros(x.shape)
        loss = torch.max(margin - (at - x), zeros)
        loss = loss**2
        loss = loss.sum() / b - margin**2

        return loss

In [ ]:
cuda = torch.cuda.is_available()
device = torch.device('cuda' if cuda else 'cpu')

model = CapsNet(A=64, B=8, C=16, D=16, E=2, K=3, P=2, iters_routings=3).to(device)
criterion = SpreadLoss(num_class=2, m_min=0.1, m_max=0.9)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=5e-5)

In [ ]:
def Accuracy(output, target, topk=(1,)):
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))

    return res

def exp_lr_decay(optimizer, global_step, init_lr = 1e-4, decay_steps = 20000, decay_rate = 0.96, lr_clip = 1e-4, ext_decay = 1, staircase=False):
    if staircase:
        lr = (init_lr * decay_rate**(global_step // decay_steps)) 
    else:
        lr = (init_lr * decay_rate**(global_step / decay_steps)) 
    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch, device):
    model.train()
    train_loss = 0
    epoch_acc = 0
    train_len = len(train_loader)
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        
        output = model(data)
        
        r = (1.*batch_idx + (epoch-1)*train_len) / (epochs*train_len)
        loss = criterion(output, target, r)
        acc = Accuracy(output, target)
        global_step = (batch_idx+1) + (epoch - 1) * len(train_loader) 
        exp_lr_decay(optimizer = optimizer, global_step = global_step)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        epoch_acc += acc[0].item()
            
    loss = train_loss / train_len
    accuracy = epoch_acc / train_len     
    
    print(f'\n Train Epoch: {epoch} \tLoss: {loss:.6f}')
    
    return loss, accuracy

In [ ]:
def test(test_loader, model, criterion, device):
    
    DNN_score = open('/data/github/result/Matrix_CapsNet_DNN_score.TXT', 'a')
     
    model.eval()
    test_len = len(test_loader)
    test_loss = 0
    acc = 0    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            
            output = model(data)
            
            if batch_idx < int(len(test_data)/batch_size):
                for n in range(0,batch_size):
                    hh=math.sqrt((output[n][1]**2).sum())
                    DNN_score.write(str(hh))
                    DNN_score.write('\n')                   
                       
            if int(len(test_data)/batch_size) == batch_idx:
                for n in range (0,int((len(test_data)/batch_size-int(len(test_data)/batch_size))*batch_size)):
                    hh=math.sqrt((output[n][1]**2).sum())
                    DNN_score.write(str(hh))
                    DNN_score.write('\n')  
                    
            test_loss += criterion(output, target, r=1).item()
            acc += Accuracy(output, target)[0].item()

    loss = test_loss / test_len
    accuracy = acc / test_len
    
    print(f'\nTest set: Loss: {loss:.6f}, Accuracy: {round(accuracy*((test_len-1)*0.2+0.1))}/{len(test_loader.dataset)}({(accuracy):.2f}%)\n')
    print('----------------------------------------------------')
    
    DNN_score.close()
    
    return loss, accuracy

In [ ]:
def Like(s, b, u, n):
    return math.e**((n*s+b)*math.log(u*s+b)-math.lgamma(n*s+b+1)-(u*s+b))
def IndiLikeRatioDis(NS1, NB1):
    return math.sqrt(-2*math.log((Like(NS1, NB1, 0.0, 1.0))/(Like(NS1, NB1, 1.0, 1.0))))

In [ ]:
DNN_score = open('/data/github/result/Matrix_CapsNet_DNN_score.TXT', 'w')    

Train_Loss = []
Train_Accuracy = []
Test_Loss = []
Test_Accuracy = []

epochs = 1
best_sig = 0

for epoch in range(1, epochs + 1):
    start=time.time()
    print('########### Training epoch {} start ###########'.format(epoch))

    train_loss, train_accuracy = train(train_loader, model, criterion, optimizer, epoch, device)
    test_loss, test_accuracy = test(test_loader, model, criterion, device)

    Train_Loss.append(train_loss)
    Train_Accuracy.append(train_accuracy)
    Test_Loss.append(test_loss)
    Test_Accuracy.append(test_accuracy)

    Results = np.loadtxt('/data/github/result/Matrix_CapsNet_DNN_score.TXT')

    epoch_ = epoch

    Results_hh_1=Results[len(test_data)*(epoch_-1):len(test_data)*(epoch_-1)+5000]
    Results_tt_1=Results[len(test_data)*(epoch_-1)+5000:len(test_data)*(epoch_-1)+5000+19000]
    Results_tw_1=Results[len(test_data)*(epoch_-1)+5000+19000:len(test_data)*(epoch_-1)+5000+19000+800]
    Results_tth_1=Results[len(test_data)*(epoch_-1)+5000+19000+800:len(test_data)*(epoch_-1)+5000+19000+800+100]
    Results_ttv_1=Results[len(test_data)*(epoch_-1)+5000+19000+800+100:len(test_data)*(epoch_-1)+5000+19000+800+100+100]
    Results_llbj_1=Results[len(test_data)*(epoch_-1)+5000+19000+800+100+100:len(test_data)*(epoch_-1)+5000+19000+800+100+100+240]
    Results_tatabb_1=Results[len(test_data)*(epoch_-1)+5000+19000+800+100+100+240:len(test_data)*epoch_]

    plt.rc('text', usetex=True)
    plt.rc('font', family='Time New Roman')

    logs = False

    axislabels = [ r'$DNN $']

    Yaxislabels = [ r'$DNN $']

    Bmax = 1
    Bmin = 0
    plt.xlim(0, 1)
    bins = np.linspace(Bmin, Bmax,  25)
    plt.hist(Results_hh_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='black', label= r'$h \; h$')
    plt.hist(Results_tt_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='blue', label= r'$t \; \overline{t}$')
    plt.hist(Results_tw_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='red', label= r'$t \; w$')
    plt.hist(Results_tth_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; h$')
    plt.hist(Results_ttv_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; v$')
    plt.hist(Results_llbj_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$l \; l \; b \; j$')
    plt.hist(Results_tatabb_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$\tau \; \tau \; b \; b$')
    plt.legend(loc=9,fontsize = 10)
    plt.xlabel(axislabels[0], fontsize = 20)
    plt.ylabel(r'$\rm{(1/\sigma) \; d \sigma / d }$' + Yaxislabels[0]    , fontsize = 20)
    plt.show()

    ROC_Results = open('/data/github/result/Matrix_CapsNet_ROC.TXT'+str(epoch), 'w')

    XSig_box = []

    Xbkg_box = []

    Xbkg_tt_box = []

    Xbkg_tw_box = []

    Xbkg_tth_box = []

    Xbkg_ttv_box = []

    Xbkg_llbj_box = []

    Xbkg_tatabb_box = []

    nn = 10000

    Ival = 0.9

    Xreco_Sig = 0.0214964

    Xreco_tt = 120.907 * 1.596

    Xreco_tw = 4.38354

    Xreco_tth = 0.15258 * 1.27

    Xreco_ttv = 0.157968 * 1.54

    Xreco_llbj = 1.22936

    Xreco_tatabb = 0.011392

    for j in range(0, nn):

        roc_sig = 0
        roc_bkg_tt = 0
        roc_bkg_tw = 0
        roc_bkg_tth = 0
        roc_bkg_ttv = 0
        roc_bkg_llbj = 0
        roc_bkg_tatabb = 0

        for i in range(0, len(Results_hh_1)):
            if Results_hh_1[i] > float( Ival + float(0.1*j)/float(nn) ) :
                roc_sig = roc_sig + 1

        for i in range(0, len(Results_tt_1 )):
            if Results_tt_1[i] > float( Ival + float(0.1*j)/float(nn) ) :
                roc_bkg_tt = roc_bkg_tt + 1

        for i in range(0, len(Results_tw_1) ):
            if Results_tw_1[i] > float( Ival + float(0.1*j)/float(nn) ) :
                roc_bkg_tw = roc_bkg_tw + 1

        for i in range(0, len(Results_tth_1 )):
            if Results_tth_1[i] > float( Ival + float(0.1*j)/float(nn) ) :
                roc_bkg_tth = roc_bkg_tth + 1

        for i in range(0, len(Results_ttv_1) ):
            if Results_ttv_1[i] > float( Ival + float(0.1*j)/float(nn) ) :
                roc_bkg_ttv = roc_bkg_ttv + 1

        for i in range(0, len(Results_llbj_1 )):
            if Results_llbj_1[i] > float( Ival + float(0.1*j)/float(nn) ) :
                roc_bkg_llbj = roc_bkg_llbj + 1

        for i in range(0, len(Results_tatabb_1) ):
            if Results_tatabb_1[i] > float( Ival + float(0.1*j)/float(nn) ) :
                roc_bkg_tatabb = roc_bkg_tatabb + 1

        XSig_box.append( float( float(Xreco_Sig)*float( roc_sig ) / float( len(Results_hh_1) ) )   )

        Xbkg_box.append( float( float(Xreco_tt)*float( roc_bkg_tt ) / float( len(Results_tt_1) ) ) + float( float(Xreco_tw)*float( roc_bkg_tw ) / float( len(Results_tw_1) ) ) + float( float(Xreco_tth)*float( roc_bkg_tth ) / float( len(Results_tth_1) ) ) + float( float(Xreco_ttv)*float( roc_bkg_ttv ) / float( len(Results_ttv_1) ) ) + float( float(Xreco_llbj)*float( roc_bkg_llbj ) / float( len(Results_llbj_1) ) ) + float( float(Xreco_tatabb)*float( roc_bkg_tatabb ) / float( len(Results_tatabb_1) ) )   )

        Xbkg_tt_box.append( float( float(Xreco_tt)*float( roc_bkg_tt ) / float( len(Results_tt_1) ) )  )

        Xbkg_tw_box.append( float( float(Xreco_tw)*float( roc_bkg_tw ) / float( len(Results_tw_1) ) )   )

        Xbkg_tth_box.append( float( float(Xreco_tth)*float( roc_bkg_tth ) / float( len(Results_tth_1) ) )  )

        Xbkg_ttv_box.append( float( float(Xreco_ttv)*float( roc_bkg_ttv ) / float( len(Results_ttv_1) ) )   )

        Xbkg_llbj_box.append( float( float(Xreco_llbj)*float( roc_bkg_llbj ) / float( len(Results_llbj_1) ) )  )

        Xbkg_tatabb_box.append( float( float(Xreco_tatabb)*float( roc_bkg_tatabb ) / float( len(Results_tatabb_1) ) )   )    

    for j in range(0, len(XSig_box) ):

        if float( Xbkg_box[j] ) == 0 :
            break

        Nsig = round( float( 3000*XSig_box[j]*(0.8**2/0.7**2) ), 3)

        Nbkg = round( float( 3000*Xbkg_box[j]*(0.8**2/0.7**2) ), 3)

        Nbkg_tt = round( float( 3000*Xbkg_tt_box[j]*(0.8**2/0.7**2) ), 3)

        Nbkg_tw = round( float( 3000*Xbkg_tw_box[j]*(0.8**2/0.7**2) ), 3)

        Nbkg_tth = round( float( 3000*Xbkg_tth_box[j]*(0.8**2/0.7**2) ), 3)

        Nbkg_ttv = round( float( 3000*Xbkg_ttv_box[j]*(0.8**2/0.7**2) ), 3)

        Nbkg_llbj = round( float( 3000*Xbkg_llbj_box[j]*(0.8**2/0.7**2) ), 3)

        Nbkg_tatabb = round( float( 3000*Xbkg_tatabb_box[j]*(0.8**2/0.7**2) ), 3)

        SobSqrtB = round( float( IndiLikeRatioDis(float( Nsig ),float( Nbkg ) )  ) ,   3 )
        ROC_Results.write(str(Nsig) + ' ' + str(Nbkg) + ' ' + str(SobSqrtB) + ' ' + str(Nbkg_tt) + ' ' + str(Nbkg_tw) + ' ' + str(Nbkg_tth) + ' ' + str(Nbkg_ttv) + ' ' + str(Nbkg_llbj) + ' ' + str(Nbkg_tatabb)   ) 
        ROC_Results.write('\n')

    ROC_Results.close()

    ROC_Results= np.loadtxt('/data/github/result/Matrix_CapsNet_ROC.TXT'+str(epoch))

    SB=[]
    hh=[]
    tt=[]
    tw=[]
    tth=[]
    ttv=[]
    llbj=[]
    tatabb=[]

    for n in range(len(ROC_Results)):  
        SB.append(ROC_Results[n][2])
        hh.append(ROC_Results[n][0])
        tt.append(ROC_Results[n][3])
        tw.append(ROC_Results[n][4])
        tth.append(ROC_Results[n][5])
        ttv.append(ROC_Results[n][6])
        llbj.append(ROC_Results[n][7])
        tatabb.append(ROC_Results[n][8])

    plt.plot(hh,SB, color='r', label='Significance')
    plt.xlabel(r'$ N_s $', fontsize=20)
    plt.ylabel(r'Significance', fontsize=20)
    plt.legend(loc='best', fontsize=15)
    plt.show()

    j=SB.index(max(SB))
    print('\nsignificance: {:.3f} hh: {:.3f} tt: {:.3f} tw: {:.3f} tth: {:.3f} ttv: {:.3f} llbj: {:.3f} tatabb: {:.3f} \n'.format(SB[j], hh[j], tt[j], tw[j], tth[j], ttv[j], llbj[j], tatabb[j]))         

    if epoch % 1 == 0:
        sig = max(SB)
        best_sig = max(best_sig,sig)           

    end=time.time()

    print('* Best Significance : {:.3f} *'.format(best_sig))

    print('Epoch time: {:.2f} mins'.format((end-start)/60))
    print('='*69)      